In [1]:
# Scope object

import cv2
import easyocr
import sys
import os
import pandas as pd
from ipywidgets import widgets, Image, Output
from PIL import Image as Pilimage
from PIL import ImageOps
import numpy as np

class scope():
    def __init__(self, saved_string=None):
        self.reader = easyocr.Reader(['ch_sim','en'])
        
        self.size = [512,512]
        
        self.output = Output()
        self.slider_h = widgets.IntRangeSlider(max=self.size[0], value=(315,450),layout=widgets.Layout(width='90%'),observe=self._update_preview)
        self.slider_v = widgets.IntRangeSlider(max=self.size[1], value=(90,140),layout=widgets.Layout(width='90%'),observe=self._update_preview)
        
        self.minus_comp = widgets.Checkbox(value=False,observe=self._update_preview)
        
        self.minus_brightness = widgets.Text(value="150", layout=widgets.Layout(width='90%',observe=self._update_preview))
        self.minus_label = widgets.Label(value="Brightness value to validate minus (0-255)")
                                             
        self.offset_x = widgets.IntSlider(max=320//2, min=320//-2, value=0, layout=widgets.Layout(width='90%'),observe=self._update_preview)
        self.offset_y = widgets.IntSlider(max=320//2, min=320//-2, value=0, layout=widgets.Layout(width='90%'),observe=self._update_preview)
        
        self.replacement_list = widgets.Text(value="[('o','0'),('O','0'),(',','.')]",layout=widgets.Layout(width='90%'),observe=self._update_preview)
        
        self.rota = widgets.Dropdown(options=['0', '90'], value='0', description='Rotation:', layout=widgets.Layout(width='90%'), observe=self._update_preview)
        self.flip = widgets.Checkbox(value=False,observe=self._update_preview)
        self.flop = widgets.Checkbox(value=False,observe=self._update_preview)
        
        self.cropped_display = widgets.Image()
        
        self.slider_h.observe(self._update_preview)
        self.slider_v.observe(self._update_preview)
        self.minus_comp.observe(self._update_preview)
        # self.minus_brightness(self._update_preview)
        self.offset_x.observe(self._update_preview)
        self.offset_y.observe(self._update_preview)
        self.rota.observe(self._update_preview)
        self.flip.observe(self._update_preview)
        self.flop.observe(self._update_preview)
        
        # Tabbed view for simpler controls
        tab_contents = ['Crop', 'Offset']
        children = [widgets.VBox([
            widgets.Label(value="Width:"),
            self.slider_h,
            widgets.Label(value="Height:"),
            self.slider_v
        ]),widgets.VBox([
            widgets.Label(value="Use minus correction"),
            self.minus_comp,
            self.minus_label,
            self.minus_brightness,
            widgets.Label(value="Offset X:"),
            self.offset_x,
            widgets.Label(value="Offset Y:"),
            self.offset_y
        ]),widgets.VBox([
            widgets.Label(value="replacement list:"),
            self.replacement_list
        ]),widgets.VBox([
            widgets.Label(value="Image rotation:"),
            self.rota,
            widgets.Label(value="Flip:"),
            self.flip,
            widgets.Label(value="Flop:"),
            self.flop
        ])]
        
        tab = widgets.Tab()
        tab.children = children
        tab.titles = ['Crop', 'Minus Comp', 'Replacement List', 'Edit Image']
        
        self._update_preview()
        
        display(
            tab,
            widgets.Label(value="Out:"),
            self.cropped_display, 
            self.output,
        )
        
        # TODO:
        # Do we need some kind of security key here maybe?
        # Only signed strings?
        if saved_string != None:
            exec(saved_string)
        
    def _update_preview(self, event=0):
        self._update_image()
        location_size = 10
        location_color = (255,255,0)
        # Crop the display image out of the full capture
        self.cropped = self.image.crop((self.slider_h.value[0],
                                        self.slider_v.value[0],
                                        self.slider_h.value[1],
                                        self.slider_v.value[1]))
        # draw cross
        if self.minus_comp.value:
            x_pos = (self.cropped.size[0]//2-1)+self.offset_x.value
            y_pos = (self.cropped.size[1]//2-1)+self.offset_y.value
            cropped_cv2 = np.asarray(self.cropped)
            #Somthing has changed in PIL image as before I needed to convert colorspace, now the image seems to be RGB natively
            #cropped_cv2 = cv2.cvtColor(np.asarray(self.cropped), cv2.COLOR_BGR2RGB)

            self.edited = cv2.line(cropped_cv2,(x_pos,y_pos-location_size),(x_pos,y_pos+location_size),location_color,2)
            self.edited = cv2.line(self.edited,(x_pos-location_size,y_pos),(x_pos+location_size,y_pos),location_color,2)

            self.cropped_display.value = Pilimage.fromarray(self.edited, mode='RGB')._repr_png_()
            
            pixels = self.cropped.load()
            cropped_x = (self.slider_h.value[1]-self.slider_h.value[0])/2 + self.offset_x.value
            cropped_y = (self.slider_v.value[1]-self.slider_v.value[0])/2 + self.offset_y.value
            # Average over 3 pixels
            average = (pixels[cropped_x-1,cropped_y][0] + pixels[cropped_x,cropped_y][0] + pixels[cropped_x+1,cropped_y][0])/3
            # Display current value in UI
            self.minus_label.value = f"Brightness value to validate minus (0-255/{round(average)})"
            self.minus_label.layout.display = 'none'
            self.minus_label.layout.display = 'flex'
        else:
            self.cropped_display.value = self.cropped._repr_png_()
        self.cropped_display.format = 'PNG'
    
    def _update_image(self):
        camera = cv2.VideoCapture(cam)
        _, img = camera.read()            
        try:
            self.origImg = img
            img = Pilimage.fromarray(img)
        except Exception as e:
            raise ValueError("[\033[31mERROR\033[0m] The camera returned None image, check if the camera connection works, if necessary restart the camera")
            # return
        self.image = img
        
        self.image = img.rotate(int(self.rota.value), expand=True)
        if self.flip.value:
            self.image = ImageOps.mirror(img)
        if self.flop.value:
            self.image = ImageOps.flip(img)

        if self.rota.value == '90':
            self.slider_h.max = img.size[1]
            self.slider_v.max = img.size[0]
        else:
            self.slider_h.max = img.size[0]
            self.slider_v.max = img.size[1]
        
        camera.release()
        
    def get_val(self):
        # value as text
        val_text = self.reader.readtext(np.asarray(self.cropped))[0][1]
        # make the replacements
        elements = eval(self.replacement_list.value)
        for element in elements:
            val_text = val_text.replace(element[0], element[1])
        
        if val_text == "Under":
            raise ValueError("[\033[31mERROR\033[0m] The lamp has gone dark, probably something bad happened")
        
        if val_text[0:2] == "00":
            val_text = "0.0"+val_text[0:2]
        #
        try:
            val_int = eval(val_text)
        except Exception as e:
            raise ValueError("[\033[31mERROR\033[0m] Could not convert to number from: " + val_text)

        if val_int == 0 and val_text !="0.0000":
            raise ValueError("[\033[31mERROR\033[0m] Could not get the entire number from: " + val_text)
        # minus comp
        if self.minus_comp.value:
            # Translate realtive cordinates to cropped local X,Y
            pixels = self.cropped.load()
            cropped_x = (self.slider_h.value[1]-self.slider_h.value[0])/2 + self.offset_x.value
            cropped_y = (self.slider_v.value[1]-self.slider_v.value[0])/2 + self.offset_y.value
            # Average over 3 pixels
            average = (pixels[cropped_x-1,cropped_y][0] + pixels[cropped_x,cropped_y][0] + pixels[cropped_x+1,cropped_y][0])/3
            
            # Check if blue channel is over treshold
            try:
                if average > int(self.minus_brightness.value):
                    # If the output is offensively positive
                    if val_int>0:
                        val_int = val_int * -1
            except Exception as e:
                pass
                
        return(val_int)
    
    def update(self):
        # Try for 6 times to get an understandable value from input image use 10 second cooldow
        sucess = False
        for i in range(7):
            try:
                self._update_preview()
                self.lastVal = self.get_val()
                sucess = True
                break
            except Exception as e:
                print(e)
                print(f"Using 10s cooldown and will try again {i+1}/6")
                time.sleep(10)
        
        if sucess:
            return(self.lastVal)
        else:
            playFailure()
            raise ValueError("[\033[31mERROR\033[0m] Could get value even after several tries, just giving up")
    
    def save_string(self):
        # Used to print string to init new object with saved properties
        construction_string = ""
        construction_string = construction_string + "self.slider_h.value = "+str(self.slider_h.value)+"\n"
        construction_string = construction_string + "self.slider_v.value = "+str(self.slider_v.value)+"\n"
        construction_string = construction_string + "self.minus_comp.value = "+str(self.minus_comp.value)+"\n"
        construction_string = construction_string + "self.minus_brightness.value = '"+str(self.minus_brightness.value)+"'\n"
        construction_string = construction_string + "self.offset_x.value = "+str(self.offset_x.value)+"\n"
        construction_string = construction_string + "self.offset_y.value = "+str(self.offset_y.value)+"\n"
        construction_string = construction_string + "self.rota.value = '"+str(self.rota.value)+"'\n"
        construction_string = construction_string + "self.flip.value = "+str(self.flip.value)+"\n"
        construction_string = construction_string + "self.flop.value = "+str(self.flop.value)+"\n"
        construction_string = construction_string + "self.replacement_list.value =str("+str(self.replacement_list.value)+") \n"
        return construction_string
        
        
    

In [12]:
import serial
import time

KelvinRangeIds = [2800,3200,4800,5600,7800,10000]
TargetCodeWarm = [0,78,156,313,625,1250,2500,5000,10000] # Used for 2800-3200
TargetCodeCool = [0, 156,313,625,1250,2500,5000,10000, 20000] # Used for 4800-10000

class apollo_device():
    """Represents an Apollo device with communication and local variables
    
        TODO:
        - [ ] Fix Json
        - [ ] Bring calib into object
        - [ ] Finish dockstrings
    """

    def __init__(self, ComPort = 'COM18', ComBaudRate=115200, SekBrightnes=None, SekDuv=None, SekKelvin=None):
        """
        Create a new device instance with dedicated serial port and feedback Scope Objects for automated calibration and testing

        Args:
        - ComPort (str): The port identifier.
        - ComBaudRate (int): Baud rate for communication.
        - SekBrightnes: Scope Object for brightness measurements.
        - SekDuv: Scope Object for Duv measurements.
        - SekKelvin: Scope Object for Kelvin measurements.
        """
        self.port = ComPort
        self.portBaudrate = ComBaudRate

        # Initialize serial connection
        self.ser = serial.Serial(self.port, self.portBaudrate, timeout=0.1)
        
        # Buffer to store received characters
        self.receive_buffer = ""
        
        # Sekonic reader objects cropped to single value
        self.SekBrightnes = SekBrightnes
        self.SekDuv = SekDuv
        self.SekKelvin = SekKelvin

        self.ser.reset_input_buffer()
        # Current color properties
        self.color = [0,0,0,0,0]
        self.macId = ""
        #self.getId()
        #self.update()
        
    def send(self, command):
        """
        Send a barebone write command through the serial connection. Kind of like alias to make it look like other objects I am used to.

        Args:
        - command (bytes): The command to be sent as bytes.
        
        TODO:
        - Remove from all other code and then from here
        """
        self.ser.write(command)
        
    def update(self, intensityRed=None, intensityGreen=None, intensityBlue=None, intensityWhite=None, fan=None, debugFlag=False, rgbtMode=False):
        """
        Update is called to sync local object to real world object, write the data to lamp
        Usually update is called as:
            lamp1.color = (100,100,100,100,0)
            lamp1.update()
            
        Parameters are used for debuging for last minute overrides
        RGBt mode is the same function as DMX input

        Args:
        - intensityRed (int): Red intensity value (0-255 in RGBt).
        - intensityGreen (int): Green intensity value (0-255 in RGBt).
        - intensityBlue (int): Blue intensity value (0-255 in RGBt).
        - intensityWhite (int): White balance range value (0-255 in RGBt).
        - fan (int): Fan intensity.
        - debugFlag (bool): Will print sent data and recieved data.
        - rgbtMode (bool): Flag for RGBt/DMX mode.
        """
        if intensityRed == None:
            intensityRed = self.color[0]
        if intensityGreen == None:
            intensityGreen = self.color[1]
        if intensityBlue == None:
            intensityBlue = self.color[2]
        if intensityWhite == None:
            intensityWhite = self.color[3]
        if fan == None:
            fan=self.color[4]

        # A mode 2048 values no calib (RGBW)
        messageStart = 'A'
        
        # I mode only 8 bit inputs are supported (RGBt)
        if rgbtMode:
            messageStart = 'I'
        
        message = messageStart+' '\
                  +str(intensityRed)+' '\
                  +str(intensityGreen)+' '\
                  +str(intensityBlue)+' '\
                  +str(intensityWhite)+' '\
                  +str(fan)+'\n'
        
        self.send(message.encode('utf-8'))
        if debugFlag:
            print(message.encode('utf-8'))

        self.read(debugFlag=debugFlag)


    def getId(self, debugFlag=False):
        """
        Get MAC id from ESP32 chip in the lamp
        
        Args:
        - debugFlag (bool): Will print sent data and recieved data.
        """
        message = "M\n"
        self.send(message.encode('utf-8'))
        if debugFlag:
            print(message.encode('utf-8'))

        self.read(debugFlag=debugFlag)
        self.macId = eval("{"+self.lastValAsJsonString[1:-1]+"}")['mac']
        
    def read(self, debugFlag=False):
        """
        Read serial port to recieve data from the lamp
        
        ToDo:
        - Look At the retry system, seems wierd
        
        Args:
        - debugFlag (bool): Will print sent data and recieved data.
        """
        #self.ser.reset_input_buffer()
        #time.sleep(0.1)
        retryCount = 50;
        noPackageCount = 0;
        while noPackageCount<retryCount:
            # Read available characters
            data = self.ser.read()

            if data:
                self.receive_buffer = self.receive_buffer + data.decode('utf-8', errors='replace')
                    
                if data == b'\n':
                    if debugFlag:
                        print(self.receive_buffer)
                        
                    timeInMs = time.time_ns() # 1_000_000
                    timeAsStr = str(timeInMs)
                    self.lastValAsJsonString = "{"+self.receive_buffer[self.receive_buffer.find('"'):-2]+',"Timestamp":'+timeAsStr+"}"
                    self.receive_buffer = ""
                    noPackageCount = 50;
            else:
                time.sleep(0.01)
                noPackageCount = noPackageCount+1
                
    def AdjustLux(self, CurBrightnessId, CurKelvinId, lutIn, sleepMult=1.):
        """
        Automated calibration function for adjusting brightness to desired level
        
        Args:
        - CurBrightnessId (int): 1-8 (2800-3200)[78,156,313,625,1250,2500,5000,10000] (4800-10000)[156,313,625,1250,2500,5000,10000,20000]
        - CurKelvinId (int): 0-5 [2800,3200,5600,7800,10000]
        - lutIn (int[]): calibration_point list
        - sleepMult(float): to adjust longer or shorter timeouts to accomadate different calibartion devices
        """
        # do five measurtements untill direction is changed, then return the best value
        # list of last 5 measurtements
        self.luxLast = []
        if CurKelvinId > 1:
            brightnessIdeal = TargetCodeCool[CurBrightnessId]
        else:
            brightnessIdeal = TargetCodeWarm[CurBrightnessId]

        whitepoint = [
            lutIn[CurKelvinId][8][2],
            lutIn[CurKelvinId][8][3],
            lutIn[CurKelvinId][8][4],
            lutIn[CurKelvinId][8][5]
            ]
        
        currentPoint = [
            lutIn[CurKelvinId][CurBrightnessId][2],
            lutIn[CurKelvinId][CurBrightnessId][3],
            lutIn[CurKelvinId][CurBrightnessId][4],
            lutIn[CurKelvinId][CurBrightnessId][5]
            ]
        
        # Sleep time logic
        sleep = 3 * sleepMult
        if CurBrightnessId < 4:
            sleep = 6 * sleepMult
        if CurBrightnessId < 2:
            sleep = 9 * sleepMult
        
        # Get the second smallest item in the list, divide all numbers by the this
        # multpily the resulted float by 10 to get large step
        qualifiedValues = [x for x in whitepoint if x != 0]
        devider = sorted(qualifiedValues)[1]
        
        self.luxStepSmall = [round(element / devider) for element in whitepoint]
        self.luxStepLarge = [round(element / (devider / 10)) for element in whitepoint]
        
        print(f'Adjusting lamp brightness to {brightnessIdeal}lx in {KelvinRangeIds[CurKelvinId]}K with {self.luxStepSmall}/{self.luxStepLarge} steps')
        for i in range(6):
            # measure current state if there is no previous measurement
            if self.luxLast == []:
                # set color and seleep
                self.color = currentPoint + [0]
                self.update()
               
                # Collect the answer from the lamp
                # LUT will need the red_value that is actually output after RED calibration
                self.read()
                jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
                jsonObject = eval(jsonString)
                redActual = jsonObject['red_val']

                # Measure result
                time.sleep(sleep)
                color_raw = [redActual, self.color[1], self.color[2], self.color[3], self.color[4]]
                curBrightness = self.SekBrightnes.update()
                diff = brightnessIdeal - curBrightness
                self.luxLast.append([curBrightness, diff] + self.color)
                print(f'current brightness: {curBrightness}, ideal {brightnessIdeal}, diff {diff} : {self.color} / {color_raw}')

                # if it is best it possibly can, why to continue
                if diff == 0:
                    break
            
            # Set Step Size
            if abs(diff) < 200:
                self.luxStep = self.luxStepSmall
            else:
                self.luxStep = self.luxStepLarge
            # Set Step Direction
            if diff < 0:
                self.luxStep = [element * -1 for element in self.luxStep]

            # Update lamp with the Step
            steppedColor = [x + y for x, y in zip(self.color, self.luxStep+[0])]
            # clip outside 0-2048
            for ch in range(len(steppedColor)):
                if steppedColor[ch] > 2047:
                    steppedColor[ch] = 2048
                if steppedColor[ch] < 1:
                    steppedColor[ch] = 0   
            self.color = steppedColor
            self.update()
            
            # Collect the answer from the lamp
            # LUT will need the red_value that is actually output after RED calibration
            self.read()
            jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
            jsonObject = eval(jsonString)
            redActual = jsonObject['red_val']
            
            # Measure result
            time.sleep(sleep)
            color_raw = [redActual, self.color[1], self.color[2], self.color[3], self.color[4]]
            curBrightness = self.SekBrightnes.update()
            diff = brightnessIdeal - curBrightness
            self.luxLast.append([curBrightness, diff] + self.color)
            print(f'current brightness: {curBrightness}, ideal {brightnessIdeal}, diff {diff} : {self.color} / {color_raw}')
            
            # if it is best it possibly can, why to continue
            if diff == 0:
                break
        # get the best possible result from the pass and write this out
        # Find the row with the value in the second column closest to 0
        min_row = min(self.luxLast, key=lambda x: abs(x[1]))
        lutIn[CurKelvinId][CurBrightnessId][2] = min_row[2]
        lutIn[CurKelvinId][CurBrightnessId][3] = min_row[3]
        lutIn[CurKelvinId][CurBrightnessId][4] = min_row[4]
        lutIn[CurKelvinId][CurBrightnessId][5] = min_row[5]
        print("selected: " + str(lutIn[CurKelvinId][CurBrightnessId]))
        return lutIn
    
    def AdjustDuv(self, CurBrightnessId, CurKelvinId, lutIn, sleepMult=1.):
        # do five measurtements untill direction is changed, then return the best value
        # list of last 5 measurtements
        self.duvLast = []
        
        currentPoint = [
            lutIn[CurKelvinId][CurBrightnessId][2],
            lutIn[CurKelvinId][CurBrightnessId][3],
            lutIn[CurKelvinId][CurBrightnessId][4],
            lutIn[CurKelvinId][CurBrightnessId][5]
            ]
        
        # Sleep time logic
        sleep = 3 * sleepMult
        if CurBrightnessId < 4:
            sleep = 6 * sleepMult
        if CurBrightnessId < 2:
            sleep = 9 * sleepMult
        
        # Get the second smallest item in the list, divide all numbers by the this
        # multpily the resulted float by 10 to get large step
        
        self.duvStepSmall = [0, 1, 0, 0]
        self.duvStepLarge = [0, 30, 0, 0]
        
        print(f'Adjusting lamp dUV to +/- 0.003 in {KelvinRangeIds[CurKelvinId]}K with {self.duvStepSmall}/{self.duvStepLarge} steps')
        for i in range(6):
            # measure current state if there is no previous measurement
            if self.duvLast == []:
                # set color and seleep
                self.color = currentPoint + [0]
                self.update()
                # Collect the answer from the lamp
                # LUT will need the red_value that is actually output after RED calibration
                self.read()
                jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
                jsonObject = eval(jsonString)
                redActual = jsonObject['red_val']

                # Measure result
                time.sleep(sleep)
                color_raw = [redActual, self.color[1], self.color[2], self.color[3], self.color[4]]
                diff = self.SekDuv.update()
                self.duvLast.append([diff, diff] + self.color)
                print(f'current dUV: {diff}, ideal 0.0000, diff {diff} : {self.color} / {color_raw}')

                # if it is best it possibly can, why to continue
                if diff == 0:
                    break
            
            # Set Step Size
            self.duvStep = self.duvStepSmall
            if abs(diff) > 0.0005 and CurBrightnessId > 6:
                if CurKelvinId > 0:
                    self.duvStep = self.duvStepLarge
            if abs(diff) > 0.002 and CurBrightnessId > 3:
                if CurKelvinId > 2:
                    self.duvStep = self.duvStepLarge

                
            # Set Step Direction
            if diff > 0:
                self.duvStep = [element * -1 for element in self.duvStep]

            # Update lamp with the Step
            steppedColor = [x + y for x, y in zip(self.color, self.duvStep+[0])]
            # clip outside 0-2048
            for ch in range(len(steppedColor)):
                if steppedColor[ch] > 2047:
                    steppedColor[ch] = 2048
                if steppedColor[ch] < 1:
                    steppedColor[ch] = 0
            self.color = steppedColor
            self.update()
            
            # Collect the answer from the lamp
            # LUT will need the red_value that is actually output after RED calibration
            self.read()
            jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
            jsonObject = eval(jsonString)
            redActual = jsonObject['red_val']
            
            # Measure result
            time.sleep(sleep)
            color_raw = [redActual, self.color[1], self.color[2], self.color[3], self.color[4]]
            diff = self.SekDuv.update()
            self.duvLast.append([diff, diff] + self.color)
            print(f'current dUV: {diff}, ideal 0.0000, diff {diff} : {self.color} / {color_raw}')
            
            # if it is best it possibly can, why to continue
            if diff == 0:
                break
        # get the best possible result from the pass and write this out
        # Find the row with the value in the second column closest to 0
        min_row = min(self.duvLast, key=lambda x: abs(x[1]))
        lutIn[CurKelvinId][CurBrightnessId][2] = min_row[2]
        lutIn[CurKelvinId][CurBrightnessId][3] = min_row[3]
        lutIn[CurKelvinId][CurBrightnessId][4] = min_row[4]
        lutIn[CurKelvinId][CurBrightnessId][5] = min_row[5]
        print("selected: " + str(lutIn[CurKelvinId][CurBrightnessId]))
        return lutIn
    
    def AdjustKelvin(self,CurBrightnessId, CurKelvinId, lutIn, sleepMult=1.):
        # do five measurtements untill direction is changed, then return the best value
        # list of last 5 measurtements
        self.kelvinLast = []
        
        currentPoint = [
            lutIn[CurKelvinId][CurBrightnessId][2],
            lutIn[CurKelvinId][CurBrightnessId][3],
            lutIn[CurKelvinId][CurBrightnessId][4],
            lutIn[CurKelvinId][CurBrightnessId][5]
            ]
        
        kelvinIdeal = KelvinRangeIds[CurKelvinId]
        
        # Sleep time logic
        sleep = 3 * sleepMult
        if CurBrightnessId < 4:
            sleep = 6 * sleepMult
        if CurBrightnessId < 2:
            sleep = 9 * sleepMult
        
        # Get the second smallest item in the list, divide all numbers by the this
        # multpily the resulted float by 10 to get large step
        
        if CurKelvinId < 2:
            self.kelvinStepSmall = [-1, 0, 0, 1]
            self.kelvinStepLarge = [-10, 0, 0, 10]
        else:
            self.kelvinStepSmall = [-1, 0, 1, 0]
            self.kelvinStepLarge = [-10, 0, 10, 0]            
        
        print(f'Adjusting lamp Kelvin to {KelvinRangeIds[CurKelvinId]}K with {self.kelvinStepSmall}/{self.kelvinStepLarge} steps')
        for i in range(6):
            # measure current state if there is no previous measurement
            if self.kelvinLast == []:
                # set color and sleep
                self.color = currentPoint + [0]
                self.update()
                # Collect the answer from the lamp
                # LUT will need the red_value that is actually output after RED calibration
                self.read()
                jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
                jsonObject = eval(jsonString)
                redActual = jsonObject['red_val']

                # Measure result
                time.sleep(sleep)
                color_raw = [redActual, self.color[1], self.color[2], self.color[3], self.color[4]]
                curKelvin = self.SekKelvin.update()
                diff = kelvinIdeal - curKelvin
                self.kelvinLast.append([curKelvin, diff] + self.color)
                print(f'current Kelvin: {curKelvin}K, ideal {kelvinIdeal}K, diff {diff}K : {self.color} / {color_raw}')
                
                # if it is best it possibly can, why to continue
                if diff == 0:
                    break
            
            # Set Step Size
            self.kelvinStep = self.kelvinStepSmall
            if abs(diff) > 200 and CurBrightnessId > 5:
                self.kelvinStep = self.kelvinStepLarge
            if abs(diff) > 1500 and CurBrightnessId > 4:
                self.kelvinStep = self.kelvinStepLarge

            # Set Step Direction
            if diff < 0:
                self.kelvinStep = [element * -1 for element in self.kelvinStep]

            # Update lamp with the Step
            steppedColor = [x + y for x, y in zip(self.color, self.kelvinStep+[0])]
            # clip outside 0-2048
            for ch in range(len(steppedColor)):
                if steppedColor[ch] > 2047:
                    steppedColor[ch] = 2048
                if steppedColor[ch] < 1:
                    steppedColor[ch] = 0
            self.color = steppedColor
            self.update()
            # Collect the answer from the lamp
            # LUT will need the red_value that is actually output after RED calibration
            self.read()
            jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
            jsonObject = eval(jsonString)
            redActual = jsonObject['red_val']
            
            # Measure result
            time.sleep(sleep)
            color_raw = [redActual, self.color[1], self.color[2], self.color[3], self.color[4]]
            curKelvin = self.SekKelvin.update()
            diff = kelvinIdeal - curKelvin
            self.kelvinLast.append([curKelvin, diff] + self.color)
            print(f'current Kelvin: {curKelvin}K, ideal {kelvinIdeal}K, diff {diff}K : {self.color} / {color_raw}')
            
            # if it is best it possibly can, why to continue
            if diff == 0:
                break
        # get the best possible result from the pass and write this out
        # Find the row with the value in the second column closest to 0
        min_row = min(self.kelvinLast, key=lambda x: abs(x[1]))
        lutIn[CurKelvinId][CurBrightnessId][2] = min_row[2]
        lutIn[CurKelvinId][CurBrightnessId][3] = min_row[3]
        lutIn[CurKelvinId][CurBrightnessId][4] = min_row[4]
        lutIn[CurKelvinId][CurBrightnessId][5] = min_row[5]
        print("selected: " + str(lutIn[CurKelvinId][CurBrightnessId]))
        return lutIn
        
    def close(self):
        self.ser.close()
        
    def color_limit(self):
        # Define the shared channel limit
        channel_limit = (0, 2047)  

        # Create a list to store the fixed values
        fixed_color = [min(max(value, channel_limit[0]), channel_limit[1]) for value in self.color]

        # Update the original color with the fixed values
        self.color = tuple(fixed_color) 
        
def add_vector5(v_one, v_two):
    out = (v_one[0]+v_two[0],v_one[1]+v_two[1],v_one[2]+v_two[2],v_one[3]+v_two[3],v_one[4]+v_two[4])
    return out

def split_vector4to_vector5(v_one, fanVal):
    out = (int(v_one[0]/2),int(v_one[1]/2),int(v_one[2]/2),int(v_one[3]/2),fanVal)
    return out

def remap(old_val, old_min, old_max, new_min, new_max):
    """
    Arduino map like Remap a value from an old range to a new range.

    Args:
    - old_val (float): Value in the old range.
    - old_min (float): Minimum value in the old range.
    - old_max (float): Maximum value in the old range.
    - new_min (float): Minimum value in the new range.
    - new_max (float): Maximum value in the new range.

    Returns:
    - float: Remapped value in the new range.
    """
    return (new_max - new_min)*(old_val - old_min) / (old_max - old_min) + new_min





Calibration starting points

In [8]:
KelvinRangeIds = [2800,3200,4800,5600,7800,10000]
# Base Calibration object
calibration_points = [[[2800, 0, 0, 0, 0, 0], [2800, 1, 9, 3, 0, 4], [2800, 2, 18, 9, 0, 7], [2800, 3, 37, 18, 0, 14], [2800, 4, 74, 38, 0, 28], [2800, 5, 148, 79, 1, 56], [2800, 6, 295, 164, 2, 112], [2800, 7, 590, 329, 4, 224], [2800, 8, 1181, 668, 9, 449]], [[3200, 0, 0, 0, 0, 0], [3200, 1, 8, 3, 0, 5], [3200, 2, 15, 8, 0, 10], [3200, 3, 30, 15, 0, 19], [3200, 4, 61, 34, 0, 38], [3200, 5, 122, 72, 1, 76], [3200, 6, 244, 144, 2, 152], [3200, 7, 488, 295, 4, 303], [3200, 8, 976, 596, 9, 606]], [[4800, 0, 0, 0, 0, 0], [4800, 1, 6, 3, 0, 18], [4800, 2, 11, 6, 0, 36], [4800, 3, 22, 14, 1, 72], [4800, 4, 44, 31, 2, 144], [4800, 5, 88, 66, 4, 287], [4800, 6, 186, 145, 8, 564], [4800, 7, 372, 292, 15, 1127], [4800, 8, 838, 664, 17, 2047]], [[5600, 0, 0, 0, 0, 0], [5600, 1, 4, 2, 1, 18], [5600, 2, 8, 6, 2, 36], [5600, 3, 17, 15, 4, 73], [5600, 4, 39, 37, 9, 139], [5600, 5, 80, 82, 18, 276], [5600, 6, 168, 176, 36, 544], [5600, 7, 337, 352, 71, 1087], [5600, 8, 705, 691, 130, 2047]], [[7800, 0, 0, 0, 0, 0], [7800, 1, 4, 4, 3, 17], [7800, 2, 7, 9, 6, 34], [7800, 3, 14, 20, 11, 68], [7800, 4, 30, 45, 22, 134], [7800, 5, 59, 96, 44, 269], [7800, 6, 118, 194, 89, 538], [7800, 7, 235, 379, 178, 1077], [7800, 8, 479, 741, 345, 2047]], [[10000, 0, 0, 0, 0, 0], [10000, 1, 4, 4, 4, 16], [10000, 2, 7, 11, 8, 33], [10000, 3, 15, 25, 16, 63], [10000, 4, 28, 52, 32, 129], [10000, 5, 54, 107, 63, 260], [10000, 6, 105, 214, 126, 523], [10000, 7, 196, 395, 251, 1061], [10000, 8, 386, 760, 485, 2047]]]

In [6]:
import pyperclip

# beam function to flatten object to b64 string that can unflatten itself in python
def beam(
    obj, 
    name='obj', 
    git='https://github.com/KKallas/Jupyter-Periscope', 
    file='Jupyter-Serializer.ipynb',
    desc='Python native imports'):
    """
    beam
    
    stringfy any python object to base64 string so it can be emailed, imd or functionally created
    
    obj: obje to strringify
    name: (optional, default 'obj') name of the restored variable
    """
    ### IMPORTS
    # Python native imports
    import codecs
    # A little failsafe to pip install dill if system does not have it yet
    try:
        import dill
    except e as Exception:
        !pip install dill
        import dill
        
    ### LOGIC
    # I merged few line here sorry, just dilling the object and encoding to base64 without any new line so it would not take multiple lines when pasted into new workbook
    nonewlines = codecs.encode(dill.dumps(obj),"base64").decode().replace('\n','')
    template = "#  git: " + git + "\n# file: " + file + "\n# desc: " + desc + "\n# A little failsafe to pip install dill if system does not have it yet\nimport codecs\ntry:\n    import dill\nexcept e as Exception:\n    !pip install dill\n    import dill\npayload = \'\'\'"+nonewlines+"\'\'\'\nw."+name+" = dill.loads(codecs.decode(payload.encode(), 'base64'))"
    
    return template

# register new functionality to the [w] menu system
def beam(obj, name='obj'):
    pyperclip.copy(beam(obj, name=name))
    print("step copied...")
    

In [2]:
import pygame

def playSucess():
    pygame.mixer.init()
    pygame.mixer.music.load('phrase_3.mp3')
    pygame.mixer.music.play()
    
def playFailure():
    pygame.mixer.init()
    pygame.mixer.music.load('phrase_4.mp3')
    pygame.mixer.music.play()


pygame 2.1.2 (SDL 2.0.18, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
playFailure()